In [2]:
import pandas as pd
import numpy as np
from scipy.stats import linregress
from sklearn.preprocessing import MinMaxScaler

# 출력 옵션 설정
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [3]:
# -----------------------------------------------------------
# 1. 데이터 로드
# -----------------------------------------------------------
# 파일 경로 - 개별 설정
file_path_usage = "../../30만원본/GENERAL/300k_general_승인정보.csv"
file_path_balance = "../../30만원본/GENERAL/300k_general_잔액정보.csv"

# 필요한 컬럼 지정 (메모리 효율화)
cols_usage = [
    '발급회원번호', '기준년월',
    '이용금액_신용_B0M', '이용건수_신용_B0M',  # 기울기 점수용
    '승인거절건수_B0M',                      # 리스크 점수용
    '이용개월수_전체_R3M', '이용개월수_전체_R6M'  # 종합이탈점수(활동성)용
]

cols_balance = [
    '발급회원번호', '기준년월',
    '잔액_B0M',                             # 기울기 점수용
    '연체잔액_B0M',                         # 리스크/연체강도 점수용
    '잔액_현금서비스_B0M', '잔액_현금서비스_B1M', # 악성부채
    '잔액_카드론_B0M', '잔액_카드론_B1M',       # 악성부채
    '연체원금_B1M', '연체원금_B2M',             # 연체강도
    '평잔_3M', '평잔_6M'                      # 자산방어
]

print("데이터 로드 중...")
# encoding='cp949'는 한글 윈도우 환경에서 흔히 사용됨
try:
    # usecols=lambda x: x in cols_... 방식을 쓰거나 리스트를 직접 전달
    # 리스트 전달 시 파일에 없는 컬럼이 있으면 에러가 나므로, lambda 방식이 안전할 수 있으나
    # 여기서는 정확한 분석을 위해 필요한 컬럼이 없으면 에러를 확인하는 것이 좋음.
    # 단, 승인거절건수 등의 위치가 확실치 않으면 일단 로드 시도.
    
    # 주의: 승인거절건수_B0M 컬럼이 실제 파일에 있는지 확인 필요.
    # 만약 에러 발생 시 컬럼명 확인 후 수정해야 함.
    df_usage = pd.read_csv(file_path_usage, usecols=lambda c: c in cols_usage)
    df_balance = pd.read_csv(file_path_balance, usecols=lambda c: c in cols_balance)
    
    print(f"승인정보 로드 완료: {df_usage.shape}")
    print(f"잔액정보 로드 완료: {df_balance.shape}")
    
except ValueError as e:
    print("데이터 로드 중 에러 발생 (컬럼명 불일치 등):", e)
    raise

데이터 로드 중...
승인정보 로드 완료: (240636, 7)
잔액정보 로드 완료: (240636, 12)


In [4]:
# -----------------------------------------------------------
# 2. 데이터 병합
# -----------------------------------------------------------
df = pd.merge(df_usage, df_balance, on=['발급회원번호', '기준년월'], how='inner')
df.fillna(0, inplace=True)
print(f"병합 완료: {df.shape}")

병합 완료: (240636, 17)


In [5]:
# -----------------------------------------------------------
# 3. [모델 1] 기울기 기반 이탈 점수 (Slope Churn Score)
# -----------------------------------------------------------
# 3-1. 데이터 정렬
df = df.sort_values(by=['발급회원번호', '기준년월'])

# 3-2. 이동 기울기 함수
def calc_slope(y):
    if len(y) < 2 or np.sum(y) == 0:
        return 0.0
    x = np.arange(len(y))
    slope, _, _, _, _ = linregress(x, y)
    return slope

group = df.groupby('발급회원번호')

print("기울기 계산 중 (시간이 조금 걸릴 수 있습니다)...")
df['Slope_Spend'] = group['이용금액_신용_B0M'].rolling(window=3, min_periods=2).apply(calc_slope).reset_index(level=0, drop=True)
df['Slope_Balance'] = group['잔액_B0M'].rolling(window=3, min_periods=2).apply(calc_slope).reset_index(level=0, drop=True)
df['Slope_Count'] = group['이용건수_신용_B0M'].rolling(window=3, min_periods=2).apply(calc_slope).reset_index(level=0, drop=True)

df[['Slope_Spend', 'Slope_Balance', 'Slope_Count']] = df[['Slope_Spend', 'Slope_Balance', 'Slope_Count']].fillna(0)

# 3-3. 기울기 -> 점수 변환 (MinMax Scaling, 감소폭이 클수록 위험)
scaler = MinMaxScaler()
norm_scores = {}
for col in ['Slope_Spend', 'Slope_Balance', 'Slope_Count']:
    # 음수(감소)인 경우만 추출하여 양수로 변환 (가장 많이 줄어든 게 가장 큰 값 = 위험)
    neg_val = df[col].apply(lambda x: -x if x < 0 else 0)
    norm_scores[col] = scaler.fit_transform(neg_val.values.reshape(-1, 1)).flatten()

# 3-4. 가중치 적용 및 Score 1 산출
W_SPEND, W_BALANCE, W_COUNT, W_RISK = 40, 30, 20, 10

# Risk Flag: 연체나 거절이 있으면 1점
# (승인거절건수_B0M 컬럼이 존재한다고 가정)
if '승인거절건수_B0M' in df.columns:
    refusal_cnt = df['승인거절건수_B0M']
else:
    refusal_cnt = 0

df['Risk_Flag'] = np.where(
    (df['연체잔액_B0M'] > 0) | (refusal_cnt > 0),
    1, 0
)

df['Churn_Score_Slope'] = (
    (norm_scores['Slope_Spend'] * W_SPEND) +
    (norm_scores['Slope_Balance'] * W_BALANCE) +
    (norm_scores['Slope_Count'] * W_COUNT) +
    (df['Risk_Flag'] * W_RISK)
)
df['Churn_Score_Slope'] = df['Churn_Score_Slope'].round(1)

print("Slope Score 계산 완료")

기울기 계산 중 (시간이 조금 걸릴 수 있습니다)...
Slope Score 계산 완료


In [6]:
df.head()

,기준년월,발급회원번호,이용건수_신용_B0M,이용금액_신용_B0M,이용개월수_전체_R6M,이용개월수_전체_R3M,승인거절건수_B0M,잔액_B0M,잔액_현금서비스_B0M,잔액_카드론_B0M,연체잔액_B0M,잔액_현금서비스_B1M,잔액_카드론_B1M,연체원금_B1M,연체원금_B2M,평잔_3M,평잔_6M,Slope_Spend,Slope_Balance,Slope_Count,Risk_Flag,Churn_Score_Slope
13465,201807,SYN_1000048,28,640723,6,3,0,557562,0,0,0,0,0,0,0,847762,1472905,0.0,0.0,0.0,0,0.0
53571,201808,SYN_1000048,25,553131,6,3,0,592750,0,0,0,0,0,0,0,795867,1183289,-87592.0,35188.0,-3.0,0,4.2
93677,201809,SYN_1000048,27,593691,6,3,0,547562,0,0,0,0,0,0,0,721781,1090827,-23516.0,-5000.0,-0.5,0,0.9
133783,201810,SYN_1000048,33,548379,6,3,0,394581,0,0,0,0,0,0,0,465095,1028880,-2376.0,-99084.5,4.0,0,0.3
173889,201811,SYN_1000048,34,592141,6,3,0,596195,0,0,0,0,0,0,0,379381,409493,-775.0,24316.5,3.5,0,0.0


In [7]:
# -----------------------------------------------------------
# 4. [모델 2] 종합 이탈 점수 (Total Churn Score)
# -----------------------------------------------------------

# [부정 1] 점수_악성부채
df['점수_악성부채'] = (
    ((df['잔액_현금서비스_B0M'] - df['잔액_현금서비스_B1M']) / (df['잔액_현금서비스_B1M'] + 1) * 1.5) +
    ((df['잔액_카드론_B0M'] - df['잔액_카드론_B1M']) / (df['잔액_카드론_B1M'] + 1) * 1.0)
)

# [부정 2] 점수_연체강도
df['점수_연체강도'] = (
    (3.0 * df['연체잔액_B0M']) + 
    (2.0 * df['연체원금_B1M']) + 
    (1.0 * df['연체원금_B2M'])
)

# [긍정 1] 점수_활동성 (높을수록 좋음 -> 나중에 뺌)
df['점수_활동성'] = (
    ((df['이용개월수_전체_R3M'] * 2) - df['이용개월수_전체_R6M']) / (df['이용개월수_전체_R6M'] + 1) * 100
)

# [긍정 2] 점수_자산방어 (높을수록 좋음 -> 나중에 뺌)
df['점수_자산방어'] = (
    df['평잔_3M'] / (df['평잔_6M'] + 1) * 10
)

# 종합 이탈 점수 (Raw) 계산
df['Total_Churn_Raw'] = (
    (df['점수_악성부채'] + df['점수_연체강도']) - 
    (df['점수_활동성'] + df['점수_자산방어'])
)

# -----------------------------------------------------------
# 5. 점수 통합 (Ensemble)
# -----------------------------------------------------------
# 5-1. 모델 2 점수 스케일링 (0~100)
scaler_total = MinMaxScaler(feature_range=(0, 100))
df['Churn_Score_Total_Scaled'] = scaler_total.fit_transform(df[['Total_Churn_Raw']])

# 5-2. 최종 점수 산출 (5:5 가중치)
df['Final_Combined_Score'] = (
    (df['Churn_Score_Slope'] * 0.5) +
    (df['Churn_Score_Total_Scaled'] * 0.5)
)
df['Final_Combined_Score'] = df['Final_Combined_Score'].round(1)

print("최종 통합 점수 산출 완료")

최종 통합 점수 산출 완료


In [8]:
# -----------------------------------------------------------
# 6. 결과 확인
# -----------------------------------------------------------
target_month = df['기준년월'].max()
df_final_view = df[df['기준년월'] == target_month].copy()

cols_view = [
    '발급회원번호', '기준년월', 'Slope_Spend', 'Slope_Balance', 'Slope_Count', 'Risk_Flag',
    'Churn_Score_Slope', 'Churn_Score_Total_Scaled',
    'Final_Combined_Score'
]

print(f"\n[{target_month} 기준 최종 이탈 점수 상위 20명]")
print(df_final_view[cols_view].sort_values('Final_Combined_Score', ascending=False).head(20))

# 저장 (옵션)
# df[cols_view].to_csv("final_churn_score_result.csv", index=False, encoding='utf-8')


[201812 기준 최종 이탈 점수 상위 20명]
             발급회원번호    기준년월  Slope_Spend  Slope_Balance  Slope_Count  Risk_Flag  Churn_Score_Slope  Churn_Score_Total_Scaled  Final_Combined_Score
213323   SYN_951536  201812          0.0      -548838.5         -1.0          1               12.3                 28.137529                  20.2
230701  SYN_2268131  201812          0.0     -1104833.5         -1.0          1               13.9                 23.230863                  18.6
214216  SYN_1015214  201812     -94645.5       184921.5         -1.5          1               13.3                 23.086684                  18.2
202533   SYN_149668  201812          0.0       -16987.0          0.0          1               10.0                 24.915494                  17.5
210400   SYN_737051  201812          0.0     -1462582.5         -2.5          1               16.0                 18.499923                  17.2
203930   SYN_255979  201812          0.0        -3701.0         -3.0          1          

In [13]:
df_final_view[cols_view]

,발급회원번호,기준년월,Slope_Spend,Slope_Balance,Slope_Count,Risk_Flag,Churn_Score_Slope,Churn_Score_Total_Scaled,Final_Combined_Score
213995,SYN_1000048,201812,101549.5,292358.5,1.0,0,0.0,5.749713,2.9
213996,SYN_1000194,201812,2276.0,-73453.5,-2.0,0,1.6,5.749736,3.7
213997,SYN_1000201,201812,-54752.0,-24431.0,-2.5,0,3.1,5.749688,4.4
213998,SYN_1000250,201812,-28865.5,-832.0,0.0,0,0.7,5.749732,3.2
213999,SYN_1000338,201812,-53005.5,-26227.5,0.0,0,1.3,5.749710,3.5
...,...,...,...,...,...,...,...,...,...
201862,SYN_99965,201812,-50289.0,-13824.5,-0.5,0,1.6,5.749595,3.7
213991,SYN_999664,201812,122892.0,168926.0,-0.5,0,0.3,5.749736,3.0
213992,SYN_999749,201812,27204.0,-148841.0,-3.5,0,2.8,5.749735,4.3
213993,SYN_999765,201812,-35216.0,-2034.0,-0.5,0,1.2,5.749691,3.5


In [14]:
df_final_view['Churn_Score_Total_Scaled'].value_counts(normalize=True)

Churn_Score_Total_Scaled
5.749777    0.241360
5.750105    0.018850
5.750215    0.002693
5.750269    0.002668
5.749710    0.000050
              ...   
5.749742    0.000025
5.749736    0.000025
5.749739    0.000025
5.749723    0.000025
5.749726    0.000025
Name: proportion, Length: 29458, dtype: float64